# Imports

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
from mpl_toolkits.axes_grid1 import Size, Divider

# Read data

In [ ]:
folder = 'plot_data/'
filename = 'rec_line_cons_equ_bed_hreg2.nc'

with xr.open_dataset(folder + filename) as ds:
    dataset = ds
    true_bed_h  = ds.total_true_bed_h.data
    true_sfc_h = ds.true_surface_h.data
    guessed_bed_h = ds.guessed_bed_h.data
    guessed_sfc_h = ds.surface_h.data
    total_distance = ds.coords['total_distance'].data
    ice_mask = ds.ice_mask.data
    ice_distance = total_distance[ice_mask]
    first_guess_bed_h = ds.first_guessed_bed_h.data
    first_guess_sfc_h = ds.first_guess_surface_h.data

In [ ]:
dataset

# start plot for exploring data

In [ ]:
figsize=(20,14)
fig = plt.figure(figsize=figsize, facecolor='white')
ax = fig.subplots()

ax.plot(total_distance, true_bed_h, '.-', label='true bed')
ax.plot(total_distance, true_sfc_h, '.-', label='true sfc')
#ax.plot(ice_distance, first_guess_bed_h, label='first guess')
#ax.plot(total_distance, first_guess_sfc_h, label='first guess')

#for i in np.arange(0,5):
#    ax.plot(ice_distance, guessed_bed_h[i], label=str(i) + '. Iteration')
#    ax.plot(total_distance, guessed_sfc_h[i], label=str(i) + '. Iteration')

ax.plot(ice_distance, guessed_bed_h[1], '.-', label=str(len(guessed_bed_h)) + '. Iteration')
ax.plot(total_distance, guessed_sfc_h[1], '.-', label=str(len(guessed_bed_h)) + '. Iteration')
ax.legend(fontsize=25)
ax.set_xlim([5.5,6.5])
ax.set_ylim([2340, 2800])

show evolution:
- first row: first guess, 1. Iteration, 2. Iteration , 3. Iteration
- second row: 10. Iteration, 20. Iteration, 30. Iteration, 40. Iteration

# Define colors

In [ ]:
colors = sns.color_palette("colorblind")
colors

In [ ]:
true_bed_color = list(colors[3]) + [1.] #list(colors[3]) + [1.]
second_bed_color = list(colors[0]) + [1.]
glacier_color = list(colors[9]) + [.5]
outline_color = [0., 0., 0., 1.]
axis_color = list(colors[7]) + [1.]

# subfigure plot

In [ ]:
def subplot(
            ax,
            true_bed_h,
            true_sfc_h,
            second_bed_h,
            second_sfc_h,
            label,
            fontsize,
            lw=2,
            ms=20,
            add_legend=False):
    # define index which points should be used
    index = np.arange(56, 66)
    
    # plot true bed_h and surface_h
    ax.plot(total_distance[index],
            true_bed_h[index],
            '.-',
            lw=lw,
            ms=ms,
            c=true_bed_color,
            label=r'true $b$')
    ax.plot(total_distance[index],
            true_sfc_h[index],
            '.--',
            lw=lw,
            ms=ms,
            c=true_bed_color,
            label=r'$s^{end}_o$')
    
    # plot second bed_h and surface_h
    ax.plot(total_distance[index],
            np.append(second_bed_h, true_bed_h[~ice_mask])[index],
            '.-',
            lw=lw,
            ms=ms,
            c=second_bed_color,
            zorder=5,
            label=r'guessed $b$')
    ax.plot(total_distance[index],
            second_sfc_h[index],
            '.--',
            lw=lw,
            ms=ms,
            c=second_bed_color,
            label=r'$s^{end}_m$')
    
    # add glacier polygon
    x_use=total_distance[index]
    x_polygon = np.concatenate((x_use, x_use[::-1]))
    y_polygon = np.concatenate((np.append(second_bed_h, true_bed_h[~ice_mask])[index], 
                                second_sfc_h[index][::-1]))
    coord_polygon = np.concatenate((np.expand_dims(x_polygon, axis=1),np.expand_dims(y_polygon, axis=1)), axis=1)
    ax.add_patch(Polygon(coord_polygon,
                         fc=glacier_color,  
                         ec=None,#outline_color,
                         closed=False,
                         lw = 0.8,
                         zorder=1,
                         label=''))
    
    # add labels for grid points
    tick_labels = [r'$trm$', r'$-1$', r'$-2$']
    point_indices = [63, 62, 61]
    len_label_line = 110
    text_y_distance = 2
    extra_distance_between_labels = 15
    
    for i, (tick_label, point_index)in enumerate(zip(tick_labels, point_indices)):
        # plot line
        ax.plot([total_distance[point_index],
                 total_distance[point_index]],
                [np.min([true_bed_h[point_index], second_bed_h[point_index]]),
                 true_bed_h[point_index] +
                 len_label_line +
                 i * extra_distance_between_labels],
               '-',
               c=axis_color,
               lw=lw-1,
               zorder=1)
        
        # add tick label text
        ax.text(total_distance[point_index],
                true_bed_h[point_index] +
                 len_label_line +
                 text_y_distance +
                 i * extra_distance_between_labels,
                tick_label,
                fontsize=fontsize,
                c=axis_color,
                verticalalignment='bottom',
                horizontalalignment='center')
    
    # add text with description
    ax.text(6.57, 2700,
            label,
            fontsize=fontsize,
            verticalalignment='center',
            horizontalalignment='right')
    # add visual axis
    x_origin = total_distance[index[0]]
    z_origin = true_bed_h[index[-1]]
    x_len = 0.12
    z_len = 50
    x_text_setoff = 0.015
    z_text_setoff = 10

    # add z axis
    plt.annotate(text='',
                 xy=(x_origin, z_origin), 
                 xytext=(x_origin, z_origin + z_len),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=axis_color,
                                 lw=1),
                 zorder=1
                 )
    plt.text(x_origin ,z_origin + z_len + z_text_setoff,'z',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=axis_color)

    # add x axis
    plt.annotate(text='',
                 xy=(x_origin, z_origin), 
                 xytext=(x_origin + x_len, z_origin),
                 arrowprops=dict(arrowstyle='<-',
                                 mutation_scale=25,
                                 color=axis_color,
                                 lw=1),
                 zorder=1
                 )
    plt.text(x_origin + x_len + x_text_setoff, z_origin, 'x',
             horizontalalignment='center',
             verticalalignment='center',
             fontsize=fontsize + 2,
             c=axis_color)
    # set limits of x and y axis
    ax.set_xlim([5.5,6.6])
    ax.set_ylim([2340, 2720])
    
    if add_legend:
        ax.legend(fontsize=fontsize)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
fontsize = 25
figsize=(6,5)

fig = plt.figure(figsize=figsize, facecolor='white')
ax = fig.subplots()
subplot(
    ax=ax,
    true_bed_h=true_bed_h,
    true_sfc_h=true_sfc_h,
    second_bed_h=first_guess_bed_h,
    second_sfc_h=first_guess_sfc_h,
    label='first guess',
    fontsize=fontsize,
    add_legend=False)

# legend plot

In [ ]:
def add_legend(ax,
               fontsize,
               lw,
               ms):
    # plot true bed_h and surface_h
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=true_bed_color,
            label=r'$b_{t}$')
    ax.plot([],
            [],
            '.--',
            lw=lw,
            ms=ms,
            c=true_bed_color,
            label=r'$s^{e}_{o}$')
    
    # plot second bed_h and surface_h
    ax.plot([],
            [],
            '.-',
            lw=lw,
            ms=ms,
            c=second_bed_color,
            zorder=5,
            label=r'$b$')
    ax.plot([],
            [],
            '.--',
            lw=lw,
            ms=ms,
            c=second_bed_color,
            label=r'$s^{e}_{m}$')
    
    ax.legend(loc='center', fontsize=fontsize)
    ax.axis('off')

# Create whole figure

In [ ]:
# define some parameters for ploting
lw=2
ms=20
fontsize = 25

fig = plt.figure(figsize=(1,1), facecolor='white')

# define grid
# define fixed size of subplot
subplot_width = 5
subplot_height = 5
subplot_separation_x = .1
subplot_separation_y = .1

# define height of legend
legend_height = 5

#define separation legend subplots
separation_y_legend_subplots = .1

# fixed size in inch
# along x axis                                                              x-index for locator
horiz = [Size.Fixed(subplot_width),                                       # 0 1st column subplot
         Size.Fixed(subplot_separation_x),   
         Size.Fixed(subplot_width),                                       # 2 2nd column subplot
         Size.Fixed(subplot_separation_x),
         Size.Fixed(subplot_width),                                       # 4 3rd column subplot
         Size.Fixed(subplot_separation_x),
         Size.Fixed(subplot_width),                                       # 6 4th column subplot
        ]
                                                                              # y-index for locator
vert = [Size.Fixed(subplot_height),                                       # 0 2nd row subplot
        Size.Fixed(subplot_separation_y),
        Size.Fixed(subplot_height),                                       # 2 1st row subplot
        Size.Fixed(separation_y_legend_subplots),
        Size.Fixed(legend_height)                                         # 4 legend
       ]

rect = (0., 0., 1., 1.)  # Position of the grid in the figure

# divide the axes rectangle into grid whose size is specified by horiz * vert
divider = Divider(fig, rect, horiz, vert, aspect=False)

# first guess
ax = fig.subplots()
subplot(
    ax=ax,
    true_bed_h=true_bed_h,
    true_sfc_h=true_sfc_h,
    second_bed_h=first_guess_bed_h,
    second_sfc_h=first_guess_sfc_h,
    label='(a) first guess',
    fontsize=fontsize,
    lw=lw,
    ms=ms)
ax.set_axes_locator(divider.new_locator(nx=0, ny=2))

# add iteration 1, 2, 3
for i, prefix in zip(np.arange(1,4), ['(b) ', '(c) ', '(d) ']):
    ax = fig.subplots()
    subplot(
        ax=ax,
        true_bed_h=true_bed_h,
        true_sfc_h=true_sfc_h,
        second_bed_h=guessed_bed_h[i-1],
        second_sfc_h=guessed_sfc_h[i-1],
        label=prefix + str(i) + '. Iteration',
        fontsize=fontsize,
        lw=lw,
        ms=ms)
    ax.set_axes_locator(divider.new_locator(nx=i*2, ny=2))

# add iteration 10, 20, 30
for column, (i, prefix) in enumerate(zip(np.arange(10,31,10), ['(e) ', '(f) ', '(g) '])):
    ax = fig.subplots()
    subplot(
        ax=ax,
        true_bed_h=true_bed_h,
        true_sfc_h=true_sfc_h,
        second_bed_h=guessed_bed_h[i-1],
        second_sfc_h=guessed_sfc_h[i-1],
        label=prefix + str(i) + '. Iteration',
        fontsize=fontsize,
        lw=lw,
        ms=ms)
    ax.set_axes_locator(divider.new_locator(nx=(column+1)*2, ny=0))

# add legend
ax = fig.subplots()
add_legend(ax=ax,
           fontsize=fontsize,
           lw=lw,
           ms=ms)
ax.set_axes_locator(divider.new_locator(nx=0, ny=0))

fig.savefig('instability_plot.pdf',format='pdf',bbox_inches='tight',dpi=300);